# Requests to Computer Vision APIs (plant.id, PlantNet, iNaturalist, NIA)

In [1]:
# import of needed packages
import base64
import requests
import getpass
import csv
import os
import json
from pprint import pprint
import time

In [2]:
# function to request the Plant.id API
def plantID (imageList, token, dir):
  image_data = []
  for image in imageList:
    test= readImage(dir + "/" + image) 
    image_data.append(test)
  json_data = {
    "images": image_data,
    "plant_details": ["common_names", "url", "taxonomy"]
  }
  response = requests.post(
    "https://api.plant.id/v2/identify",
    json=json_data,
    headers={
        "Content-Type": "application/json",
        "Api-Key": token
    }).json()
  result= [image]
  print(response)
  for suggestion in response["suggestions"]:
      result.append(suggestion["plant_name"])
      result.append(str(suggestion["probability"]))
  return result

In [ ]:
# function to request the plantNet API
def plantNet (image, token, dir): 
  image_data = open(dir + "/" + image[0], 'rb')
  api_endpoint = f"https://my-api.plantnet.org/v2/identify/all?api-key={token}"


  files = [
      ('images', (image[0], image_data, ), )
  ]
    
  data = {
     'organs': [image[1].lstrip()]
  }

  req = requests.Request('POST', url=api_endpoint, files=files, data=data)
  prepared = req.prepare()

  s = requests.Session()
  response = s.send(prepared)
  json_result = json.loads(response.text)

  pprint(response.status_code)
  pprint(json_result)
  result = [image[0]]
  if(response.status_code==404):
        result.append("not found")
  elif(response.status_code==200): 
      for suggestion in json_result["results"]:
          result.append(suggestion["species"]["scientificNameWithoutAuthor"]) 
          result.append(str(suggestion["score"]))
  return result

In [ ]:
# function to request an access token for the NIA (Nature Identification API)
def NiaRetrieveAccess (clientId, email, password): 
    data = {
      'client_id': clientId,
      'grant_type': 'password',
      'email': email,
      'password': password
    }

    response = requests.post('https://waarneming.nl/api/v1/oauth2/token/', data=data)

    response_text_json = json.loads(response.text) 
    acces_token = response_text_json["access_token"]

    return acces_token

In [ ]:
# function to request the NIA API
def NIA(image, token, dir):
    image_dir = dir + "/" + image 
    
    headers = {
    'Authorization': 'Bearer ' + token,
    }

    params = (
    ('app_name', 'uni-muenster'),
    )

    files = {
    'image': (image_dir, open(image_dir, 'rb')),
    }
        
    response = requests.post('https://waarneming.nl/api/identify-proxy/v1/', headers=headers, params=params, files=files)
        
    response_text_json = json.loads(response.text)
    
    result = [image]
    
    for prediction in response_text_json["predictions"]:
        result.append(prediction["taxon"]["name"])
        result.append(str(prediction["probability"])) 
  
    return result


In [ ]:
# function to request the iNaturalist API
def iNaturalist(image, token, dir):
    api_endpoint = "https://visionapi.p.rapidapi.com/v1/rapidapi/score_image"
    image_data = open(dir + "/" + image, 'rb')

    headers = {
        'x-rapidapi-host': "visionapi.p.rapidapi.com",
        'x-rapidapi-key': token
    }

    files = [
      ('image', (image, image_data))
    ]

    req = requests.Request('POST', headers=headers, url=url, files=files)
    prepared = req.prepare()
    s = requests.Session()
    response = s.send(prepared)
    json_result = json.loads(response.text)
    result = [image]
    if(response.status_code==200): 
          for suggestion in json_result["results"]:
              result.append(suggestion["taxon"]["name"])  
              result.append("n/a")
    return result

In [8]:
# encode image to base64
def readImage (image_name):
  with open(image_name, "rb") as file:
    image = [base64.b64encode(file.read()).decode("ascii")]
  return image

In [9]:
# function to write the results to csv
def writeDataToCSV (data, filename):
  with open(filename,'a+', encoding='UTF8', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(data)

In [3]:
# final excecution script
dir = input('Directory with test images: ') 
filename = input('Filename of result CSV: ')  
api = input('API ("plantNet","iNaturalist", "plantID", "NIA"):  ') 
if (api == "plantID" or api == "iNaturalist" or api=="NIA"): 
    if (api == "NIA"): 
        clientId= getpass.getpass(prompt='Client Id: ', stream=None) 
        email= input('Email: ') 
        password= getpass.getpass(prompt='Password: ', stream=None) 
        token = NiaRetrieveAccess(clientId, email, password)
    else:
        token= getpass.getpass(prompt='Api Key: ', stream=None) 
        

    # get all images from the test directory        
    images = os.listdir(dir)

    # requesting the API for each image in the test directory
    #for image in images:
    response = eval(api + "(images, token, dir)")
    writeDataToCSV(response, filename)
    if (api == "iNaturalist"):
          time.sleep(1) # limitation that just 60 requests per minute are allowed

elif (api == "plantNet"): 
    # plantNet requires an organ for each image. Organ could be one of: leaf, flower, fruit, bark
    species_list = input('Name of the CSV where corresponding organ is added to the imagefile') #csv should look like : imageName.jpg, organ \n
    with open(species_list, mode ='r')as file:
   
        # reading the CSV file
        csv_file = csv.reader(file)

        # requesting the API for each line of the CSV file
        for lines in csv_file:
                  response= plantNet(lines, api_key, dir)
                  writeDataToCSV(response, filename)
else:
   print('API must be one of "plantNet","iNaturalist", "plantID", "NIA"')

    
    

Directory with test images: plantidgeneral
Filename of result CSV: plantidgeneral.csv
API ("plantNet","iNaturalist", "plantID", "NIA"):  plantID
Api Key: ········


SSLError: HTTPSConnectionPool(host='api.plant.id', port=443): Max retries exceeded with url: /v2/identify (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: certificate has expired (_ssl.c:1091)')))